In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('all')
import re

In [ ]:
import spacy.cli
spacy.cli.download("en_core_web_lg")

In [ ]:
!pip install pymupdf

In [ ]:
import spacy
from spacy.pipeline import EntityRuler
from spacy.lang.en import English
from spacy.tokens import Doc

Reading Sample Resume

In [ ]:
import  fitz
from nltk.corpus import stopwords
stopwords=set(stopwords.words("english"))
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

fname = "/content/drive/MyDrive/Resume/Ankit_Gupta_Resume.pdf"  # get document filename
with fitz.open(fname) as doc:
   text = ""
   for page in doc:
        text += page.getText()
#print(text)
#character_only_text=re.sub("[^a-zA-Z0-9@]"," ",text)
character_only_text=re.sub("[( , )]"," ",text)
    
    
    # Lowercase and split
#lower_text=character_only_text.lower().split()
    
    #Get STOPWORDS and remove
stop_remove_text=[i for i in lower_text if not i in stopwords]

    
    #Lemmatization
lemma_removed_text=[wordnet_lemmatizer.lemmatize(word,'v') for word in stop_remove_text]
    
    
final_text= " ".join(stop_remove_text)
print("*"*50)
print(final_text)

Extracting Email and phone number using regular expression

In [ ]:
email=re.findall('\w*@[a-z]*.[a-z]*',text)
print(email)
contact_number=re.findall('\d{10}',text)
print(contact_number)

Training Custom Entity model for SKills

In [ ]:
from spacy.util import minibatch,compounding
import random
LABEL = "SKILLS"

TRAIN_DATA = [
      ("Java are too tall and they pretend to care about your feelings",
        {"entities": [(0, 4, LABEL)]},
    ),
    ("Do they bite?", {"entities": []}),
    (
        "reactJS are too tall and they pretend to care about your feelings",
        {"entities": [(0, 7, LABEL)]},
    ),
    ("HTML pretend to care about your feelings", {"entities": [(0, 4, LABEL)]}),
    (
        "they pretend to care about your feelings, those github",
        {"entities": [(48, 54, LABEL)]},
    ),
    ("Programming Language?", {"entities": [(0, 20, LABEL)]}),

    ("Software Development is my passion",{"entities":[(0,20,LABEL)]}),
              ("Programming Language is my skill",{"entities":[(0,20,LABEL)]}),
              ("c is middel level language",{"entities":[(0,1,LABEL)]}),
              ("Bootstraping ",{"entities":[(0,12,LABEL)]}),
              ("Github is for saving my repo",{"entities":[(0,6,LABEL)]}),
              ("reactjs is for app development",{"entities":[(0,7,LABEL)]}),
              ("database is used to store data",{"entities":[(0,8,LABEL)]})
]


nlp = spacy.blank("en")  # create blank Language class
ner = nlp.create_pipe("ner")
nlp.add_pipe(ner)

ner.add_label(LABEL)  # add new entity label to entity recognizer

optimizer = nlp.begin_training()

move_names = list(ner.move_names)
# get names of other pipes to disable them during training
other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]

with nlp.disable_pipes(*other_pipes):  # only train NER
    sizes = compounding(1.0, 4.0, 1.001)
    # batch up the examples using spaCy's minibatch
    for itn in range(30):
        random.shuffle(TRAIN_DATA)
        batches = minibatch(TRAIN_DATA, size=sizes)
        losses = {}
        for batch in batches:
            texts, annotations = zip(*batch)
            nlp.update(texts, annotations, sgd=optimizer, drop=0.35, losses=losses)
        print("Losses", losses)

Saving Enity pipe Model

In [ ]:
from pathlib import Path
output_dir=Path('/content/drive/MyDrive/Resume/output_dir')
if not output_dir.exists():
  output_dir.mkdir()
nlp.meta['name']='My_ner'
nlp.to_disk(output_dir)
print("saved model to ",output_dir)


Loading Custom Enity Pipe for testing

In [ ]:
nlp2=spacy.load(output_dir)
assert nlp2.get_pipe("ner")#.move_names == move_names
doc2=nlp2(text)
#for ent in doc2.ents:
 # print(ent.label_,ent.text)

Adding Custom Entity Pipe in Spacy Model

In [ ]:
import spacy
nlp = spacy.load("en_core_web_lg")
#spacy.load(output_dir)
custom_nlp = spacy.load(output_dir, vocab=nlp.vocab)
nlp.add_pipe(custom_nlp.get_pipe("ner"), name="my_ner", before="ner")

In [ ]:
Extracting Technical Skills

In [ ]:
tskills=list()
tx=nlp(text)
i=0
for l in tx.ents:
   if l.label_ == 'SKILLS' :
      tskills.append(l.text)
      print(l.text,"->",l.label_)
      

In [ ]:
print(tskills)

In [ ]:
from spacy import displacy
displacy.render(tx, style='ent',jupyter=True)

Extracting non technical Skills

In [ ]:
NTskills=list()
tx=nlp(final_text)
#print(tx.ents)
#sente = list(tx.sents)
#for s in sente:
  #print(s.ents)
  #i=0
for l in tx.ents:
   if l.label_ == ' WORK_OF_ART' or l.label_ == 'ORG' or l.label_ == 'PERSON'  :
      NTskills.append(l.text)
      print(l.text,"->",l.label_)

In [ ]:
print(NTskills)

In [ ]:
from spacy import displacy
displacy.render(tx, style='ent',jupyter=True)

Exporting data to Excel

In [ ]:
dfts = pd.DataFrame()
dfnts=pd.DataFrame()

In [ ]:
dfts['skill']=tskills[0:]
dfnts['non tec skill']=NTskills[0:]

In [ ]:
dfts.to_excel('/content/drive/MyDrive/Resume/techskills.xlsx', index = False)
dfnts.to_excel('/content/drive/MyDrive/Resume/nontechskills.xlsx', index = False)

Tried Parts of Speeach for chunking 

In [ ]:
from nltk.tokenize import word_tokenize
from nltk import ne_chunk
#tx="ankit gupta indian institute information technology vadodara email ankitgupta8292@gmail com mob 8292858632 address vill post kohara bazar dist saran bihar 841205 skills area interest software development programming language c java web technologies html css js bootstrap4 tools github visual studio code framework library reactjs redux codeigniter database postgresql mysql education degree institute year cpi aggregate b tech indian institute information technology vadodara 2017 present 7 56 intermediate shakti shanti academy dighwara chhapra 2016 76 60 high school bhagwat vidyapeeth rushi chawni chhapra 2014 9 8 cgpa projects github finder web application reactjs guide self project users search github account see details like bio followers following last repos etc project made reactjs github link project deployed netlify link hotel rooms web application reactjs guide self hotel room project customers choose room according choice project various types rooms available like double deluxe room family deluxe room presindential room etc customer see also brief details room like price capacity person size room etc made project using react js bootstrap4 technologies project made front end part github link project deployed netlify link movie series application web application reactjs redux guide self application user search movie movie related query displayed page user see brief details movie displayed page build application using reactjs redux imdb api github link library management system web application codeigniter guide self project library management admin issue book currently available library authorized member library library management admin see book issued project using html css php codeigniter framework technologies database made mysql github link online railway reservation system guide prof pm jat daiict group project dbms based postgresql users book cancel ticket also see details ticket like date journey fare ticket source station destination station github link 1 awards achievements participation coding contest codechef udacity certi cate react nanodegree program link coursera certi cate hong kong university science technology completing front end web development react course link udemy certi cate completing react course link certi cate ice technology lab completing core java course github link personal traits highly motivated eager learn new things strong motivational leadership skills ability work individual well group interests activities playing watching cricket reading newspaper listening music declaration hereby declare details furnished true best knowledge belief"
sentences = nltk.sent_tokenize(text)
for i in sentences:
  word = nltk.word_tokenize(i)
  tag = nltk.pos_tag(word)
 # NER=[ne_chunk(tag)]
  grammer="NP: {<NN|NNP><NN/NNP>}"
  chunkph=nltk.RegexpParser(grammer)
  res=chunkph.parse(tag)
  print(res)
#res.draw()

'''wt=word_tokenize(tx)
#tx=nlp(final_text)
#print(tx)
tags=nltk.pos_tag(wt)
#print(tags)
NER=[ne_chunk(sent) for sent in sentences]
print(NER)''